<a href="https://colab.research.google.com/github/avirajBevli/HP3_GDAL/blob/main/parallelize_r3_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-x0xbg8dm
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-x0xbg8dm
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=0d1aeb23f6e8b3378c82ea0e23f97bb260c9118f7d27803361c75985efa10501
  Stored in directory: /tmp/pip-ephem-wheel-cache-iyvq_zho/wheels/db/c1/1f/a2bb07bbb4a1ce3c43921252aeafaa6205f08637e292496f04
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [5]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
Fri Mar 24 22:03:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |       

In [26]:
%%cu 
#include <cuda.h>
#include <cuda_runtime.h>
#include <math.h>
 #include <iostream>
#include <bits/stdc++.h>
#include <fstream>


using namespace std;
const double INF_ = 1e9;
const double epsilon = 0.0000000001;

typedef vector<double> vd;
typedef vector<vd> vvd;
typedef vector<bool> vb;
typedef vector<vb> vvb;
typedef vector<int> vi;
typedef vector<vi> vvi;
typedef pair<int,int> ii;
typedef vector<ii> vii;



__host__ __device__ 
double calc_dist(ii p1, ii p2){
    double x1=p1.first; double y1=p1.second;
    double x2=p2.first; double y2=p2.second;
    double dx = x2-x1; double dy = y2-y1;
    return sqrt((dx*dx) + (dy*dy));
}


__host__ __device__ 
bool isvalid(ii p, int n, int m, double dist_from_origin, double range){
    if(p.first < 0 || p.second < 0) return 0;
    if(p.first >= n || p.second >= m) return 0;
    if(dist_from_origin > range) return 0;
    return 1;
}



__host__ __device__ 
double find_slope(vvi& grid, ii observer, ii curr, int observer_ht){
    double dist = calc_dist(observer, curr);
    double dh = double(grid[curr.first][curr.second]) - (double(grid[observer.first][observer.second]) + (double)observer_ht);
    double m_curr = dh/dist;
    return m_curr;
}



__host__ __device__ 
vii plotPixel(int x1, int y1, int x2, int y2, int dx, int dy, int decide){
    // pk is initial decision making parameter
    // Note:x1&y1,x2&y2, dx&dy values are interchanged
    // and passed in plotPixel function so
    // it can handle both cases when m>1 & m<1
    int pk = 2 * dy - dx;
    vii line;
    for (int i = 0; i <= dx; i++) {
        // cout << x1 << "," << y1 << endl;
        // checking either to decrement or increment the
        // value if we have to plot from (0,100) to (100,0)
        x1 < x2 ? x1++ : x1--;
        if (pk < 0) {
            // decision value will decide to plot
            // either  x1 or y1 in x's position
            if (decide == 0) {
                // putpixel(x1, y1, RED);
                line.push_back(make_pair(x1,y1));
                pk = pk + 2 * dy;
            }
            else {
                //(y1,x1) is passed in xt
                // putpixel(y1, x1, YELLOW);
                line.push_back(make_pair(y1,x1));
                pk = pk + 2 * dy;
            }
        }
        else {
            y1 < y2 ? y1++ : y1--;
            if (decide == 0) {
                line.push_back(make_pair(x1,y1));
                // putpixel(x1, y1, RED);
            }
            else {
                line.push_back(make_pair(y1,x1));
                //  putpixel(y1, x1, YELLOW);
            }
            pk = pk + 2 * dy - 2 * dx;
        }
    }
    return line;
}



__host__ __device__ 
vii bresenham(int x1, int y1, int x2, int y2){
    int dx = abs(x2 - x1);
    int dy = abs(y2 - y1);
    // If slope is less than one
    if (dx > dy) {
        // passing argument as 0 to plot(x,y)
        return plotPixel(x1, y1, x2, y2, dx, dy, 0);
    }
 
    // if slope is greater than or equal to 1
    else {
        // passing argument as 1 to plot (y,x)
        return plotPixel(y1, x1, y2, x2, dy, dx, 1);
    }
}





void readfile_and_make_grid(vvi &grid){
    //std::ifstream infile("/Users/avirajbevli/Desktop/Sem10/HPPP/cuda_term project/data/cdnf43w/heights.txt");
    cout<<"Reading from input file..."<<endl;
    std::ifstream infile("/content/heights.txt");
    if(!infile.is_open()) {
        std::cerr << "Error: Could not open read file" << std::endl;
        return  ;
    }

    string line;
    while(std::getline(infile, line)) {
        vi temp;
        string str; 
        int i=0;
        while(i<line.size()){
            if(line[i]==','){
                temp.push_back(stoi(str));
                str="";
                i++;
            }
            else{
                str+=line[i];
            }
            i++;
        }
        grid.push_back(temp);
    }
    infile.close();
    cout<<"Done reading"<<endl;
}






// -1: out of range, 0: not visible, 1: visible
__global__
void calculate_viewshed_and_writefile_r3(int * grid,int n,int m, int range){
    
   
    // parameters to the algo
    ii observer = make_pair(1000,3000);
    int observer_ht = 10; // height of observer from the ground at <observer_x, observer_y>  

    vvi viewshed(n,vi(m,-1));

    int li = max(0, (observer.first) - range);
    int ri = min(n,(observer.first) + range);
    int lj = max(0, (observer.second) - range);
    int rj = min(m,(observer.second) + range);
    cout<<"li:"<<li<<",ri:"<<ri<<", lj:"<<lj<<",rj:"<<rj<<endl;


    //minimum I = observer.first - range, maximum I = observer.first + range
    //minimum J = observer.second - range, maximum J = observer.S +range
    int I = observer.first + (blockIdx.y * blockDim.y + threadIdx.y  - range); 
    int J =  observer.second + (blockIdx.x * blockDim.x + threadIdx.x - range);

   

   if(I<li&&I>=li&&J>=lj&&J<rj)
   {
      ii curr = make_pair(I,J);
      viewshed[curr.first][curr.second]=1;
   }
      __syncthreads();

 
   if(I<li&&I>=li&&J>=lj&&J<rj)
     {
       ii target = make_pair(I,J);
       double m_target = find_slope(grid, observer, target, observer_ht); 
       vii line = bresenham(observer.first, observer.second, target.first, target.second);
       // do something with the obtained line
      
       viewshed[target.first][target.second]=1; // target is visible by default
     }
  
     double m_max = -INF_;
            int linesz = line.size();
            for(int lid=1;lid<linesz-1;lid++){
                // ii curr = line[lid];
                double m_curr = find_slope(grid, observer, line[lid], observer_ht);
                m_max = max(m_max, m_curr);
                // if(m_curr >= m_max){
                //     m_max = m_curr;
                // }
                // else{
                //     viewshed[curr.first][curr.second]=0;
                // }
            }
            if(m_max > m_target + epsilon){
                viewshed[target.first][target.second]=0;
            }
       __syncthreads();


    cout<<"Writing to output viewshed file ...."<<endl;
    std::ofstream outFile("viewshed_r3.txt");   
    if (!outFile.is_open()) {
        std::cout << "Unable to open write file" << std::endl;
        return;
    }

    for(int i=0;i<viewshed.size();i++){
        for(int j=0;j<viewshed[i].size();j++){
            outFile<<viewshed[i][j]<<" ";
        }
        outFile<<endl;
    }
    // Close the file
    outFile.close();
    cout<<"Done writing"<<endl; 
}






int main(){
    
   vvi grid;

  
   readfile_and_make_grid(grid);

  
   int n = grid.size();
   int m = grid[0].size();


  // allocate memory of grid in host RAM 
  int *grid_h;
  cudaMallocHost((void **) &grid_h, sizeof(int)*n*m);


  //copy grid to grid_h
  for(int i =0;i<n;i++)
     {
      for(int j=0;j<m;j++)
          grid_h[i*m+j] = grid[i][j];
     }


  //allocate memory to 2d vector of length n*m on device
   int *grid_d;
   cudaMalloc((void **) &grid_d, sizeof(int)*n*m);
 

  cudaMemcpy(grid_d, grid_h, sizeof(int)*n*m, cudaMemcpyHostToDevice);

  int range = 400;

   //  allocating a 2 dim kernel sizes 
    int gridCols = ceil(((2*range) +32)/32);
	  int gridRows = ceil(((2*range) +32)/32);
    
    dim3 gridDim(gridCols, gridRows,1);
  	dim3 blockDim(32, 32,1); 
    
    // Kernel call
    calculate_viewshed_and_writefile_r3<<<gridDim, blockDim>>>(grid_d,n,m,range);  
    cudaDeviceSynchronize();  
  

    return 0;
}

/tmp/tmpymoqddsx/069c3c13-4fe3-4161-879b-a97561d2fbf2.cu(71): warning #20013-D: calling a constexpr __host__ function("make_pair") from a __host__ __device__ function("plotPixel") is not allowed. The experimental flag '--expt-relaxed-constexpr' can be used to allow this.

/tmp/tmpymoqddsx/069c3c13-4fe3-4161-879b-a97561d2fbf2.cu(77): warning #20013-D: calling a constexpr __host__ function("make_pair") from a __host__ __device__ function("plotPixel") is not allowed. The experimental flag '--expt-relaxed-constexpr' can be used to allow this.

/tmp/tmpymoqddsx/069c3c13-4fe3-4161-879b-a97561d2fbf2.cu(84): warning #20013-D: calling a constexpr __host__ function("make_pair") from a __host__ __device__ function("plotPixel") is not allowed. The experimental flag '--expt-relaxed-constexpr' can be used to allow this.

/tmp/tmpymoqddsx/069c3c13-4fe3-4161-879b-a97561d2fbf2.cu(88): warning #20013-D: calling a constexpr __host__ function("make_pair") from a __host__ __device__ function("plotPixel") i